<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Langchain/langchain_IO_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain CrashCourse

## Set up

In [2]:
%%capture
!pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.30.2 trl==0.4.7 langchain --quiet
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency res

## Data downloads

In [3]:
!kaggle datasets download -d PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
!unzip amazon-reviews-unlocked-mobile-phones.zip

Dataset URL: https://www.kaggle.com/datasets/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
License(s): CC0-1.0
 77% 25.0M/32.6M [00:00<00:00, 107MB/s] 
100% 32.6M/32.6M [00:00<00:00, 110MB/s]


## Imports

In [19]:
import numpy as np
import pandas as pd
import torch
import transformers
import warnings

from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from pprint import pprint
from transformers import AutoTokenizer, AutoModelForCausalLM

warnings.filterwarnings("ignore")

## Load data

In [11]:
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')
df.dropna(inplace=True)

In [16]:
df.sample(5)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
8897,Apple iPhone 4 32GB (Black) - Verizon,Apple,69.98,5,It's been a year & this phone still runs great...,1.0
114305,BLU Dash JR W D141w Unlocked GSM Dual-SIM Andr...,BLU,34.99,5,excelente,0.0
320803,Samsung Galaxy J7 SM- J700H/DS GSM Smartphone-...,Samsung,208.00,4,bueno,0.0
294547,Posh Mobile Icon S510-B GSM Unlocked 4G HSDPA+...,Posh Mobile,68.00,5,Excellent phone. Very easy to use. The size is...,1.0
128182,BLU Life Pure Unlocked Phone (Black),BLU,150.00,5,Great phone no complaints....,0.0
